In [24]:
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

In [25]:
def spider_page(soup):
    df_company = {}
    df_company['Name'] = soup.find('h1',{'id':'company-name'}).text.replace(' ESG Score','')
    df_company['Ticker'] = soup.find('li',{'id':'company-ticker'}).text.replace('Ticker: ','')
    df_company['Score'] = soup.find('p',{'class':'scoreModule__score'}).text
    
    df_company['CSA_Score'] = ''.join(filter(str.isdigit, soup.find_all('g',{'class':'highcharts-label highcharts-stack-labels'})[0].text[:2]))
    df_company['Modeled_Score'] = ''.join(filter(str.isdigit, soup.find_all('g',{'class':'highcharts-label highcharts-stack-labels'})[1].text[:2]))
    
    df_company['Env_Score'] = soup.find('div',{'class':'dimention-chart1'}).find_all('li')[0].text[-2:]
    df_company['Env_Score_Industry_Mean'] = soup.find('div',{'class':'dimention-chart1'}).find_all('li')[1].text[-2:]
    df_company['Env_Score_Industry_Max'] = soup.find('div',{'class':'dimention-chart1'}).find_all('li')[2].text[-2:]
    
    df_company['Social_Score'] = soup.find('div',{'class':'dimention-chart2'}).find_all('li')[0].text[-2:]
    df_company['Social_Score_Industry_Mean'] = soup.find('div',{'class':'dimention-chart2'}).find_all('li')[1].text[-2:]
    df_company['Social_Score_Industry_Max'] = soup.find('div',{'class':'dimention-chart2'}).find_all('li')[2].text[-2:]
    
    df_company['Gov_Score'] = soup.find('div',{'class':'dimention-chart3'}).find_all('li')[0].text[-2:]
    df_company['Gov_Score_Industry_Mean'] = soup.find('div',{'class':'dimention-chart3'}).find_all('li')[1].text[-2:]
    df_company['Gov_Score_Industry_Max'] = soup.find('div',{'class':'dimention-chart3'}).find_all('li')[2].text[-2:]
    
    df_company['RPDRDARP%'] = soup.find_all('div',{'class':'imputed-score__table-second-column'})[0].get_text(strip=True).replace('\t','')
    df_company['RPDRDARP'] = soup.find_all('div',{'class':'imputed-score__table-second-column'})[1].get_text(strip=True)
    df_company['RPDCSAMIS'] = soup.find_all('div',{'class':'imputed-score__table-second-column'})[2].get_text(strip=True)
    df_company['PSRPDR'] = soup.find_all('div',{'class':'imputed-score__table-second-column'})[3].get_text(strip=True)
    df_company['ASRPD'] = soup.find_all('div',{'class':'imputed-score__table-second-column'})[4].get_text(strip=True)
    
    df_company['ADRDARP%'] = soup.find_all('div',{'class':'imputed-score__table-second-column'})[5].get_text(strip=True).replace('\t','')
    df_company['ADRDARP'] = soup.find_all('div',{'class':'imputed-score__table-second-column'})[6].get_text(strip=True)
    df_company['ADCSAMIS'] = soup.find_all('div',{'class':'imputed-score__table-second-column'})[7].get_text(strip=True)
    df_company['PSADR'] = soup.find_all('div',{'class':'imputed-score__table-second-column'})[8].get_text(strip=True)
    df_company['ASAD'] = soup.find_all('div',{'class':'imputed-score__table-second-column'})[9].get_text(strip=True)
    
    df_company['NQMA'] = soup.find_all('div',{'class':'imputed-score__table-second-column'})[10].get_text(strip=True)
    
    Comp_paths = soup.find('g',{'class':'highcharts-markers highcharts-series-0 highcharts-area-series highcharts-tracker'}).find_all('path', {'aria-label': True})
    for path in Comp_paths:
        aria_label = path['aria-label']
        parts = aria_label.split('. ')[1].split(', ')
        dimension = parts[0].replace(' ','')+'_Score'
        score = parts[1]
        df_company[dimension] = score
        
    Max_paths = soup.find('g',{'class':'highcharts-markers highcharts-series-1 highcharts-line-series highcharts-tracker'}).find_all('path', {'aria-label': True})
    for path in Comp_paths:
        aria_label = path['aria-label']
        parts = aria_label.split('. ')[1].split(', ')
        dimension = parts[0].replace(' ','')+'_IndustryMaxScore'
        score = parts[1]
        df_company[dimension] = score
    

    Mean_paths = soup.find('g',{'class':'highcharts-markers highcharts-series-2 highcharts-line-series highcharts-tracker'}).find_all('path', {'aria-label': True})
    for path in Comp_paths:
        aria_label = path['aria-label']
        parts = aria_label.split('. ')[1].split(', ')
        dimension = parts[0].replace(' ','')+'_IndustryMeanScore'
        score = parts[1]
        df_company[dimension] = score
    print(df_company['Name'],':',df_company['Score'],'Finished')    
    return df_company

In [26]:
# def wait_for_personal_info_form():
#     try:
#         # 增加等待时间
#         personal_info_form = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.ID, 'yui3-widget-hd modal-header'))
#         )
#         if personal_info_form:
#             print("Personal info form detected. Please fill out the form and press Enter to continue...")
#             input("Press Enter after completing the form...")
#             form_status = 1
#     except Exception as e:
#         # 输出当前页面的HTML
#         with open("page_source.html", "w") as f:
#             f.write(driver.page_source)
#         print("No personal info form detected. Saved page source to 'page_source.html'.")
#         print(f"Error: {e}")

In [27]:
def search_and_get_soup(Company_name):
    search_box = driver.find_element(By.CLASS_NAME, 'banner-search__input')
    search_box.clear()  # 清空搜索框
    search_box.send_keys(re.sub(r' CLASS.*', '', Company_name))
    # search_box.send_keys(Keys.RETURN)
    time.sleep(2)
    try:
        # 等待搜索结果加载并点击第一个搜索结果
        first_result = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.list-items li.item a'))
        )
        print("Element found, attempting to click.")
        first_result.click()
        print("Element clicked, waiting for page to load.")
    except Exception as e:
        print(f"No search suggestions for keyword '{Company_name}', now try to search '{Company_name.split(' ')[0]}'.")
        try:
            second_word = Company_name.split(' ')[1]
            first_word = Company_name.split(' ')[0]
            search_and_get_soup(first_word)
        except Exception as e:
            print(f"No search suggestions for keyword '{Company_name}', skipping.")
            skipped_companies[Company_name] = 'No results found'
            return None    
    
    try:
        WebDriverWait(driver, 10).until(
            lambda d: len(d.find_elements(By.CSS_SELECTOR, 'g.highcharts-series path')) > 5  # 这里假设有至少10个path元素，你可以根据实际情况调整
        )
    
        # 爬取选择后的页面内容
        content = driver.page_source
        soup = BeautifulSoup(content, 'html.parser')
        return soup
    
    except Exception as e:
        print(f"'{Company_name}'s ESG Score and underlying data are available via S&P premium channels only. , skipping.")
        skipped_companies[Company_name] = 'Need premium channels'
        print(f"Error: {e}")
        return None  



In [28]:
def main(rest_companies):
    try:
        close_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#onetrust-accept-btn-handler'))
        )
        close_button.click()
        time.sleep(2)  # 等待弹窗关闭
    except Exception as e:
        print("No cookie banner to close.")
        
    # 遍历所有公司
    for company_name in rest_companies:
        company_soup = search_and_get_soup(company_name)
        if company_soup:
            company_data = spider_page(company_soup)
            Found_companies.append(company_data)
                
    return company_soup

In [29]:
def Check_Results(result_sofar):
    if len(Found_companies) == 0:
        rc = rest_companies
        return rc,result_sofar
    
    else:
        df = pd.DataFrame(Found_companies)
        # 合并df1和df2，保留匹配的行
        merged_df = df.merge(Companies[['Ticker','Industry']], left_on='Ticker', right_on='Ticker', how='inner')
        
        # 删除全为空的列
        cleaned_df = merged_df.dropna(axis=1, how='all')
        # 将包含列表的列转换为字符串，以便可以删除重复行
        for col in cleaned_df.columns:
            if cleaned_df[col].apply(type).eq(list).any():
                cleaned_df[col] = cleaned_df[col].apply(str)
        
        # 删除重复行
        result_df = cleaned_df.drop_duplicates()
        result_df = pd.concat([result_df, result_sofar], axis=0, join='outer', ignore_index=True).drop_duplicates()
        # 合并df1和df2，标记出df1中的TickerKey在df2中是否存在
        merged_df = Companies[['Ticker','Name']].merge(result_df, left_on='Ticker', right_on='Ticker', how='left', indicator=True)
        
        # 提取在df2中找不到的行
        rc = merged_df[merged_df['_merge'] == 'left_only']['Name_x'].tolist()
        
        result_df.to_csv(result_path, index=False)
        print('Saved Result sofar')
        return rc, result_df

In [30]:
# 启动Chrome浏览器并使用指定的用户数据目录和配置文件
options = webdriver.ChromeOptions()
# options.headless = True  # 无头模式
# options.add_argument("--window-size=1198,1287")

# 增加调试输出
print("Setting up ChromeDriver service...")
service = Service(ChromeDriverManager().install())
print("Service set up complete.")

# 启动WebDriver服务
print("Starting ChromeDriver with options...")

url = 'https://www.spglobal.com/esg/solutions/data-intelligence-esg-scores'

# Companies = pd.read_csv('russell-3000.csv',skiprows=9,usecols=['Name','Ticker','Sector'])#.iloc[:10,]
Companies = pd.read_csv('ftse-allshares.csv',usecols=['Name','Ticker','Industry'])#.iloc[:10,]
result_path = 'S&P-ESG-Scores-FTSE-Allshares-sofar.csv'

try:
    result_sofar = pd.read_csv(result_path)
    # result_sofar = pd.read_csv('S&P-ESG-Scores-FTSE-Allshares-sofar.csv')
    merged_df = Companies[['Ticker','Name']].merge(result_sofar, left_on='Ticker', right_on='Ticker', how='left', indicator=True)
    rest_companies = merged_df[merged_df['_merge'] == 'left_only']['Name_x'].tolist()
    print('Use existing result')
except:
    result_sofar = pd.DataFrame({ 'Ticker': [], 'Name': []})
    rest_companies = Companies['Name'].tolist()
    print('Start form 0')
    
last_len = 0
skipped_companies = {}
Found_companies = []

for i in range(0,10):
    if len(rest_companies)==0:
        break
    try:
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)
        current_soup = main(rest_companies)
        driver.quit()
        rest_companies,result_sofar = Check_Results(result_sofar)
    except:
        driver.quit()
        rest_companies,result_sofar = Check_Results(result_sofar)
    if last_len==len(rest_companies):
        break
    last_len = len(rest_companies)
    print("还剩下",last_len,"个公司")

Setting up ChromeDriver service...
Service set up complete.
Starting ChromeDriver with options...
Use existing result
Element found, attempting to click.
No search suggestions for keyword '3I Infrastructure PLC', now try to search '3I'.
Element found, attempting to click.
Element clicked, waiting for page to load.
3i Group plc : 48 Finished
Element found, attempting to click.
Element clicked, waiting for page to load.
'4Imprint Group PLC's ESG Score and underlying data are available via S&P premium channels only. , skipping.
Error: Message: 
Element found, attempting to click.
Element clicked, waiting for page to load.
'AJ Bell PLC's ESG Score and underlying data are available via S&P premium channels only. , skipping.
Error: Message: 
No search suggestions for keyword 'Aberforth Smaller Companies Trust PLC', now try to search 'Aberforth'.
No search suggestions for keyword 'Aberforth', now try to search 'Aberforth'.
No search suggestions for keyword 'Aberforth', skipping.
'Aberforth Sm

In [35]:


# 过滤 rest_companies
filtered_rest_companies = [company for company in rest_companies if company not in set(skipped_companies.keys())]

In [36]:
filtered_rest_companies

['3I Infrastructure PLC',
 'Abrdn Asia Focus PLC',
 'Abrdn Asian Income Fund Limited',
 'Abrdn Diversified Income And Growth PLC',
 'Abrdn Equity Income Trust PLC',
 'Abrdn European Logistics Income PLC',
 'Abrdn New India Investment Trust PLC',
 'Abrdn UK Smaller Companies Growth Trust PLC',
 'Allianz Technology Trust PLC',
 'Avi Global Trust PLC',
 'Avi Japan Opportunity Trust PLC',
 'Aviva PLC',
 'BP PLC',
 'Bae Systems PLC',
 'Baillie Gifford China Growth Trust PLC',
 'Baillie Gifford European Growth Trust PLC',
 'Baillie Gifford Japan Trust PLC',
 'Baillie Gifford Shin Nippon PLC',
 'Baillie Gifford UK Growth Trust PLC',
 'Baillie Gifford Us Growth Trust PLC',
 'Bankers Investment Trust PLC',
 'Bbgi Global Infrastructure S.A.',
 'Bellevue Healthcare Trust PLC',
 'Blackrock Energy And Resources Income Trust PLC',
 'Blackrock Frontiers Investment Trust PLC',
 'Blackrock Greater Europe Investment Trust PLC',
 'Blackrock Latin American Investment Trust PLC',
 'Blackrock Smaller CO Tru

In [37]:
skipped_companies

{'4Imprint Group PLC': 'Need premium channels',
 'AJ Bell PLC': 'Need premium channels',
 'Aberforth': 'No results found',
 'Aberforth Smaller Companies Trust PLC': 'Need premium channels',
 'Aberforth Split Level Income Trust PLC': 'Need premium channels',
 'Aew': 'Need premium channels',
 'Aew UK Reit PLC': 'Need premium channels',
 'Alliance': 'Need premium channels',
 'Alliance Trust PLC': 'Need premium channels',
 'Ao World PLC': 'Need premium channels',
 'Apax': 'No results found',
 'Apax Global Alpha Limited': 'Need premium channels',
 'Aptitude': 'No results found',
 'Aptitude Software Group PLC': 'Need premium channels',
 'Artemis': 'No results found',
 'Artemis Alpha Trust PLC': 'Need premium channels',
 'Ascential PLC': 'Need premium channels',
 'Ashmore Group PLC': 'Need premium channels',
 'Ashoka': 'Need premium channels',
 'Ashoka India Equity Investment Trust PLC': 'Need premium channels',
 'Asia': 'No results found',
 'Asia Dragon Trust PLC': 'Need premium channels',
 